In [1]:
import pandas as pd
import requests
import ckanapi
import math
import re
import os
import json


In [2]:
mapping = {
    "id": "measure_id",
    "m": "measure_name",
    "it": "interval_type",
    "vt": "value_type",
    # "ytd": "year_to_date_ind",
    # "ht": "has_target_ind",
    "v": "variance",
    "yv": "year_to_date_variance",
    "bv": "budget_variance",
    "da": "decimal_accuracy", # should this be precision?
    "dd": "desired_direction",
    "c": "category",
    "ds":"data_source_notes",
    "cp": "city_perspective_note",
    "y": "year",
    "p": "period_number_in_year",
    "v": "measure_value",
    "target":"target",
    "note":"note",
    "c": "category",
    # "kw": "keywords",
}

def get_category_measures(measures, category):
    subset = []
    for m in measures:
        assert len(m["c"]) == 1, f"Measure has more than 1 category: {m['c']}"
        if m["c"][0].lower() == category.lower():
            subset.append(m)
            
    return subset


def make_measures_records(measures):
    records = []
    
    for i in measures:
        item = { **i }
        data_points = item.pop("vs")
        
        assert len(i["c"]) == 1, f"Item '{i['m']}' ({i['id']}) belongs to more than 1 category: {item['c']}"
        
        item["c"] = item["c"][0]
        
        for dp in data_points:
            r = { k: v for k, v in {**item, **dp}.items() if v == v }
            r["m"] = r["m"].replace("\n", " ")
            r["ds"] = r["ds"].replace("&amp;", "&")
            r.pop("ytd")
            r.pop("ht")
            r.pop("kw")
            if "da" in r:
                try:
                    r["da"] = int(r["da"])
                except:
                    r.pop("da")
            if "yv" in r:
                try:
                    r["yv"] = float(r["yv"])
                except:
                    r.pop("yv")
            if "bv" in r:
                try:
                    r["bv"] = float(r["bv"])
                except:
                    r.pop("bv")
            
            for original,updated in mapping.items():
                if original in r:
                    r[updated] = r.pop(original)

            records.append(r)
            
    return records

def join_narratives(records, narratives):
    new_records = []
    
    for record in records:
        r = {**record}
        
        n = narratives.get(f'{r[mapping["id"]]}')
        if n is not None and len(n)>1:
            r["notes"] = n.replace("<br />", "\n").strip("\n").strip()
            
        new_records.append(r)
    
    return new_records

def build_data_dict():
    data_dict = []
    
    for m in mapping.values():
        data_dict.append({
            "id": m,
            "type": "text",
        })
        
    for c in data_dict:
        if c["id"] in ["measure_id", "year_to_date_variance", "budget_variance", "measure_value","target"]:
            c["type"] = "float"
        # elif c["id"] in ["year_to_date_ind", "has_target_ind"]:
        #     c["type"] = "boolean"
        elif c["id"] in ["decimal_accuracy", "year", "period_number_in_year"]:
            c["type"] = "int"

        
    return  data_dict

def string_to_dict(string, pattern):
    regex = re.sub(r'{(.+?)}', r'(?P<_\1>.+)', pattern)
    values = list(re.search(regex, string).groups())
    keys = re.findall(r'{(.+?)}', pattern)
    _dict = dict(zip(keys, values))
    return _dict

def build_narratives_df(notes):
    p_map = {
        "January": 1,
        "February":2,
        "March":3,
        "April":4,
        "May":5,
        "June":6,
        "July":7,
        "August":8,
        "September":9,
        "October":10,
        "November":11,
        "December":12,
        "Spring":2,
        "Summer":3,
        "Fall":4,
        "Winter":1,
    }

    pattern1 = {"a":"^\[Quarter {period_number_in_year} {year}\]{note}$", "b":"\[Quarter \d \d{4}].*"}
    pattern2 = {"a":"^\[Annual {year}\]{note}$","b":"\[Annual \d{4}].*"}
    pattern3 = {"a":"^\[{period_number_in_year} {year}\]{note}$","b":"\[\w{3,15} \d{4}].*"}

    narratives=[]
    for k,v in notes.items():
        if len(v) > 10:
            for n in v.split('<br /><br />'):
                note = None
                nn = n.replace("<br />", "").strip()
                if re.fullmatch(pattern1["b"], nn, flags=0):
                    note = string_to_dict(nn,pattern1["a"])
                elif re.fullmatch(pattern2["b"], nn, flags=0):
                    note = string_to_dict(nn,pattern2["a"])
                    note["period_number_in_year"] = note["year"]
                elif re.fullmatch(pattern3["b"], nn, flags=0):
                    note = string_to_dict(nn,pattern3["a"])
                    note['period_number_in_year'] = p_map[note['period_number_in_year']]
                else:
                    None
                    # print("note does not match pattern:", n)

                if note:
                    note["year"] = int(note["year"])
                    note["period_number_in_year"] = int(note["period_number_in_year"])
                    note["measure_id"] = float(k)
                    narratives.append(note)

    return pd.DataFrame(narratives)

In [3]:
measures = requests.get("https://contrib.wp.intra.prod-toronto.ca/app_content/tpp_measures/").json()
notes = requests.get("https://contrib.wp.intra.prod-toronto.ca/app_content/tpp_narratives/").json()

records = make_measures_records(measures["measures"])
# records = join_narratives(tmp_records, notes)
fields = build_data_dict()

In [4]:
targets=measures["targets"][0]
df_target = pd.DataFrame()
for k, v in targets.items():
    df = pd.DataFrame(v)
    df["measure_id"] = float(k)
    df_target = df_target.append (df.rename(columns={"v":"target", "p":"period_number_in_year", "y":"year"}))
df_measure = pd.DataFrame(records)
df_measure_target = pd.merge(df_measure,df_target, how='left', on=['measure_id', 'year', 'period_number_in_year'])

In [5]:
df_measure_with_target = df_measure_target[df_measure_target['target'] == df_measure_target['target']][['measure_id', 'year', 'period_number_in_year','target']]
df_measure_with_target['matched']=True
print('total target number:', len(df_target), '\nmacthed:', len(df_measure_with_target))

total target number: 774 
macthed: 750


In [6]:
compare_df = pd.merge(df_target[['measure_id', 'year', 'period_number_in_year','target']], df_measure_with_target, how='left', on=['measure_id', 'year', 'period_number_in_year'])
df_target_wo_measure = compare_df[compare_df['matched'] != True][['measure_id','year','period_number_in_year','target_x']].rename(columns={"target_x":"target"})


In [7]:
# df_target_wo_measure

In [8]:
df_measure_wo_vs = df_measure_target.drop(columns=['year','period_number_in_year','measure_value','target']).drop_duplicates(keep='last')
df_measure_wo_vs['measure_value']=None


In [9]:
# df_measure_wo_vs


In [10]:
df_target_wo_vs = pd.merge(df_target_wo_measure,df_measure_wo_vs, how='left', on=['measure_id'])

In [11]:
# df_target_wo_vs

In [12]:
# df_target_wo_vs[df_measure_target.columns]

In [13]:
df = pd.concat([df_measure_target, df_target_wo_vs[df_measure_target.columns]])

In [14]:
len(df)

8310

In [15]:
df_narrative = build_narratives_df(notes)

In [16]:
len(df_narrative)

150

In [17]:
df_w_note = pd.merge(df,df_narrative, how='left', on=['measure_id', 'year', 'period_number_in_year'])

In [18]:
# df_w_note

In [19]:
df_w_note[['measure_id', 'year', 'period_number_in_year','measure_value', 'target', 'note']][df_w_note['note']==df_w_note['note']]

,measure_id,year,period_number_in_year,measure_value,target,note
166,1.130,2020,11,747.0,NaN,Decrease in line with national trend where Can...
337,1.120,2020,11,36.0,NaN,Decrease in line with national trend where Can...
502,1.040,2020,5,1273.6,NaN,Province wide emergency orders starting March ...
510,1.040,2021,1,1350.3,NaN,Statistics Canada rebased their figures based ...
1362,1.110,2020,10,86940.0,NaN,The Government of Canada made temporary change...
...,...,...,...,...,...,...
8038,2.330,2019,1,1320084.0,NaN,Much of the decrease between 2018 and 2019 Q1 ...
8142,2.350,2019,1,202757.0,NaN,Much of the increase between 2018 and 2019 Q1 ...
8145,2.350,2019,4,895844.0,NaN,At Riverlea Greenhouse the number of indoor pl...
8267,2.041,2020,2020,96750.0,NaN,Year-end Note: Ridership for 2020 was negative...


In [20]:
#verify some target 
# df_measure[(df_measure['measure_id']==2.12) & (df_measure['year']==2020) & (df_measure['period_number_in_year']==12)]

In [21]:
# df_measure_target[(df_measure_target['measure_id']==2.12) & (df_measure_target['year']==2020) & (df_measure_target['period_number_in_year']> 8)]

In [41]:
# load ckan connection from setting

loading_folder = os.getcwd()
setting_file = loading_folder+'\\..\\setting.json'
with open(setting_file) as f:
    setting = json.load(f)

ckan_connect = setting["ckan_qa"]
ckan = ckanapi.RemoteCKAN(ckan_connect['address'], ckan_connect['apikey'])


In [23]:
# find target package and resource and purge content, otherwise create resource
package_name='toronto-progress-portal'
res_name='Toronto progress portal - Key metrics'
res_id=None
now = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")

try:
    package=ckan.action.package_show(id=package_name)
    resources=package.pop('resources')
    # find datastore resource with same name and delete data
    for r1 in resources:
        if r1['name']==res_name: # and r1['datastore_active']:
            res_id = r1['id']
            ckan.action.datastore_delete(id=res_id, filters={}, force=True)
            ckan.action.resource_patch(id=res_id, datastore_active=True, is_preview=True, last_refreshed=now)
            ckan.action.package_patch(id=package['id'], last_refreshed=now)
            break
except:
    print('Error finding package')

if not res_id:
    try:
        r = {'package_id': package['id'], 'format': 'CSV', 'extract_job': 'Python-progress','name': res_name, 'datastore_active': True,'is_preview': True}
        res = ckan.action.datastore_create( resource=r, fields=fields, primary_key='measure_id,year,period_number_in_year', force=True)
        res_id = res['resource_id']
    except:
        print('Error creating resource')

In [24]:
df = df_w_note

In [25]:
i=0
while i < len(df):
    rec=[ { k:v for k,v in r.items() if (v == v) and (v is not None) } for r in df[i:i+1000].to_dict(orient="records") ]
    try:
        print('Try:',i,i+1000)
        progress_insert=ckan.action.datastore_upsert(id=res_id, records=rec, force=True, method='upsert')
    except Exception as e:
        print('Something is wrong:', e)
        break
    i=i+1000

if i >= len(df):
    print('Data loading completed:', len(df))
else:
    print('Data loading aborted')

Try: 0 1000
Try: 1000 2000
Try: 2000 3000
Try: 3000 4000
Try: 4000 5000
Try: 5000 6000
Try: 6000 7000
Try: 7000 8000
Try: 8000 9000
Data loading completed: 8310


In [33]:
# import datetime
# now = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")
# ckan.action.resource_patch(id=res_id, last_refreshed=now)
# package=ckan.action.package_show(id=package_name)
# resources=package.pop('resources')
resources

[{'cache_last_updated': None,
  'package_id': '351a2bbf-a2a1-434c-943c-84e07d66596d',
  'datastore_active': True,
  'id': 'de6ae758-336b-452d-bae1-ce2c48ba029c',
  'size': None,
  'format': 'CSV',
  'state': 'active',
  'hash': '',
  'description': '',
  'is_preview': True,
  'last_modified': None,
  'url_type': 'datastore',
  'mimetype': None,
  'cache_url': None,
  'extract_job': 'Python-progress',
  'name': 'Toronto progress portal - Key metrics',
  'created': '2021-06-04T15:23:50.164873',
  'url': 'https://ckanadmin0.intra.qa-toronto.ca/datastore/dump/de6ae758-336b-452d-bae1-ce2c48ba029c',
  'last_refreshed': '2021-06-07T10:29:12.787302',
  'mimetype_inner': None,
  'position': 0,
  'revision_id': 'e371bbc5-b6f4-441b-9063-01389c1cb1df',
  'resource_type': None}]

In [26]:
# # data from Pandas DataFrame df
# with_nans = df.to_dict(orient="records")
# ​
# # since np.nan != np.nan, can use it to filter. Essentially, knowing that the type is NOT a number doesn't mean both types ARE THE SAME non-number type.
# ​
# # ALTERNATIVE 1: create new list of records without NaNs using dictionary comprehension
# without_nans = []
# for r in with_nans:
# 	new_record = { k:v for k,v in r.items() if v == v }
#     without_nans.append(new_record)
    
# # ALTERNATIVE 2: can take it a step further by combining with list comprehension
# without_nans = [ { k:v for k,v in r.items() if v == v } for r in with_nans ]
# ​
# # ALTERNATIVE 3: can put it all in one line
# without_nans = [ { k:v for k,v in r.items() if v == v } for r in df.to_dict(orient="records") ]

In [27]:
# narratives=[]
# for k,v in notes.items():
#     if len(v) > 10:
#         for n in v.split('<br /><br />'):
#             note = None
#             nn = n.replace("<br />", "").strip()
#             if re.fullmatch(pattern1["b"], nn, flags=0):
#                 note = string_to_dict(nn,pattern1["a"])
#             elif re.fullmatch(pattern2["b"], nn, flags=0):
#                 note = string_to_dict(nn,pattern2["a"])
#                 note["period_number_in_year"] = note["year"]
#             elif re.fullmatch(pattern3["b"], nn, flags=0):
#                 note = string_to_dict(nn,pattern3["a"])
#                 note['period_number_in_year'] = p_map[note['period_number_in_year']]
#             else:
#                 print("note does not match pattern:", n)
#             if note:
#                 note["year"] = int(note["year"])
#                 note["period_number_in_year"] = int(note["period_number_in_year"])
#                 note["measure_id"] = float(k)
#                 narratives.append(note)
# df_narrative=pd.DataFrame(narratives)
# df_narrative

In [28]:
# Find out what narratives are not matched
df_note_no_match = pd.merge(df_narrative, df, how='left', on=['measure_id', 'year', 'period_number_in_year'])
df_note_no_match[df_note_no_match['measure_name'] != df_note_no_match['measure_name']]

,period_number_in_year,year,note_x,measure_id,measure_name,interval_type,value_type,measure_value,year_to_date_variance,desired_direction,category,data_source_notes,city_perspective_note,decimal_accuracy,budget_variance,target,note_y
50,3,2019,"On September 23rd, the City launched a new rec...",2.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51,2,2019,The City continues to see an increase in appli...,2.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,1,2019,The City of Toronto promoted the Summer Intern...,2.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,3,2019,"On September 23rd, the City launched a new rec...",2.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
